<img src='https://github.com/Ikomia-dev/notebooks/blob/main/examples/img/banner_ikomia.png?raw=true'>

# How to train YOLOv9

## Overview of YOLOv9
With the continuous evolution of computer vision technologies, YOLOv9 emerges as the latest advancement, developed by Chien-Yao Wang, I-Hau Yeh, and Hong-Yuan Mark Liao. This trio of researchers has a rich history in the field, having contributed to the development of preceding models such as YOLOv4, YOLOR, and YOLOv7.

### 🌖 Release

YOLOv9 not only continues the legacy of its predecessors but also introduces significant innovations that set new benchmarks in object detection capabilities.


YOLOv9 is an advanced object detection model that represents a significant leap forward in computer vision technology. It is the latest iteration in the "You Only Look Once" (YOLO) series, known for its high speed and accuracy in detecting objects in images.


### 🌆 Architecture and innovations:

YOLOv9 stands out due to its incorporation of Programmable Gradient Information (PGI) and the introduction of the Generalized Efficient Layer Aggregation Network (GELAN), two groundbreaking innovations designed to enhance model performance and efficiency.




### :rocket: Accuracy and performance:

| Model | Test Size | AP<sup>val</sup> | AP<sub>50</sub><sup>val</sup> | AP<sub>75</sub><sup>val</sup> | Param. | FLOPs |
| :-- | :-: | :-: | :-: | :-: | :-: | :-: |
| **YOLOv9-S** | 640 | **46.8%** | **63.4%** | **50.7%** | **7.2M** | **26.7G** |
| **YOLOv9-M** | 640 | **51.4%** | **68.1%** | **56.1%** | **20.1M** | **76.8G** |
| **YOLOv9-C** | 640 | **53.0%** | **70.2%** | **57.8%** | **25.5M** | **102.8G** |
| **YOLOv9-E** | 640 | **55.6%** | **72.8%** | **60.6%** | **58.1M** | **192.5G** |

## Setup

Please use a GPU for this tutorial.

In the menu, select "Runtime" then "Change runtime type", choose GPU in "Hardware accelerator".

Check your GPU with the following command:

In [ ]:
!nvidia-smi

First of all, you need to install Ikomia API pip package.

In [ ]:
!pip install ikomia



---


**-Google Colab ONLY- Restart runtime**

Click on the "RESTART RUNTIME" button at the end the previous window.





---

Ikomia API has already more than 300 pre-integrated algorithms but the most interesting algorithms are in [Ikomia HUB](https://github.com/Ikomia-hub).


## How to train YOLOv9 on a custom dataset

Download your dataset from your preferred tool. In this example, we use a dataset from **Roboflow** which is a great annotation platform used by many developers and companies. The dataset is exported in COCO format.

In [ ]:
import requests
import zipfile
import os


# Download the file
url = "https://universe.roboflow.com/ds/OG5R1BjHfr?key=U5vq9IGzOH"
response = requests.get(url, stream=True)
with open("roboflow.zip", "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

# Unzip the file
with zipfile.ZipFile("roboflow.zip", 'r') as zip_ref:
    zip_ref.extractall()

# Remove the zip file
os.remove("roboflow.zip")


In order to train YOLOv9 on your custom dataset, please create a new workflow from scratch.

Then you need 2 components:

1.   A COCO dataset loader which loads dataset in COCO format and convert it to an Ikomia format
2.   The YOLOv9 training algorithm which loads dataset in Ikomia format

Add these 2 previous algorithms to your workflow and then it will automagically download all algorithms from Ikomia Hub and install all the Python dependencies (the 1st time, it can take a while, be patient ! ).

Now, it's time to train your model !

In [ ]:
from ikomia.dataprocess.workflow import Workflow
import os

#----------------------------- Step 1 -----------------------------------#
# Create a workflow which will take your dataset as input and
# train a YOLOv9 model on it
#------------------------------------------------------------------------#
wf = Workflow()

#----------------------------- Step 2 -----------------------------------#
# First you need to convert the COCO format to IKOMIA format.
# Add an Ikomia dataset converter to your workflow.
#------------------------------------------------------------------------#

dataset = wf.add_task(name = "dataset_coco")

dataset.set_parameters({
    "json_file": os.getcwd()+"/train/_annotations.coco.json",
    "image_folder": os.getcwd()+"/train",
    "task":"detection",
    "output_folder": os.getcwd()+"/dataset"
})


#----------------------------- Step 3 -----------------------------------#
# Then, you want to train a YOLOv9 model.
# Add YOLOv9 training algorithm to your workflow
#------------------------------------------------------------------------#

train = wf.add_task(name="train_yolo_v9", auto_connect=True)
train.set_parameters({
    "model_name": "yolov9-c",
    "epochs": "50",
    "batch_size": "8",
    "train_imgsz": "640",
    "test_imgsz": "640",
    "dataset_split_ratio": "0.8",
    "output_folder": os.getcwd(),
})


#----------------------------- Step 4 -----------------------------------#
# Execute your workflow.
# It automatically runs all your tasks sequentially.
#------------------------------------------------------------------------#
wf.run()

## Infer YOLOv9 object detection on images

Once the training is finished, you may want to experiment the fresh model on new test images. Just use the following code to create a YOLO v8 instance
segmentation inference workflow.
Then run and test !

In [ ]:
# Set the output folder name
TIMESTAMP = # "27-02-2024T09h27m06s"

In [ ]:
from ikomia.dataprocess.workflow import Workflow

# Create your workflow for YOLO inference
wf = Workflow()

# Add YOLOv9 to your workflow
yolov9 = wf.add_task(name="infer_yolo_v9", auto_connect=True)

yolov9.set_parameters({
    "model_weight_file": os.getcwd()+ f'/{TIMESTAMP}/weights/best.pt',
    "class_file": os.getcwd()+ f'/{TIMESTAMP}/classes.yaml',
    "conf_thres": "0.2",
    "iou_thres":"0.25",
    "input_size":"640"
})

## Run and display your results

In [ ]:
from ikomia.utils.displayIO import display
from PIL import ImageShow
ImageShow.register(ImageShow.IPythonViewer(), 0)

# Apply YOLOv9 object detection on your image
# By default, YOLOv9 runs with a pre-trained model based on COCO
# To use your custom model, set the parameters in the previous cell

wf.run_on(url="https://pbs.twimg.com/ext_tw_video_thumb/1660454979298115585/pu/img/A_Jrl2uawkkDi_Kf.jpg")
# wf.run_on(path=os.getcwd()+"/test/youtube-128_jpg.rf.2723e31eec77e1ff7b73c45c625082f6.jpg")

# Get YOLOv9 image result
img_bbox = yolov9.get_image_with_graphics()

# Display in Colab
display(img_bbox)

## Run on video

This will work on local only, not on google colab

In [ ]:
from ikomia.dataprocess.workflow import Workflow
from ikomia.utils.displayIO import display
import cv2


video_path = 'Path/to/your/video.mp4' # Example: https://www.youtube.com/watch?v=EAR5jTknVOw
output_path = 'output.mp4'
# Init your workflow
wf = Workflow()

# Add object detection algorithm
detector = wf.add_task(name="infer_yolo_v9", auto_connect=True)

detector.set_parameters({
    "model_weight_file": os.getcwd()+ f'/{TIMESTAMP}/weights/best.pt',
    "class_file": os.getcwd()+ f'/{TIMESTAMP}/classes.yaml',
    "conf_thres": "0.2",
    "iou_thres":"0.25"
})

# Open the video file
stream = cv2.VideoCapture(video_path)
if not stream.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties for the output
frame_width = int(stream.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = stream.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
# The 'XVID' codec is widely supported and provides good quality
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

while True:
    # Read image from stream
    ret, frame = stream.read()

    # Test if the video has ended or there is an error
    if not ret:
        print("Info: End of video or error.")
        break

    # Run the workflow on current frame
    wf.run_on(array=frame)

    # Get results
    image_out = detector.get_output(0)
    obj_detect_out = detector.get_output(1)

    # Convert the result to BGR color space for displaying
    img_out = image_out.get_image_with_mask_and_graphics(obj_detect_out)
    img_res = cv2.cvtColor(img_out, cv2.COLOR_RGB2BGR)

    # Save the resulting frame
    out.write(img_out)

    # Display
    display(img_res, title="YOLOv9 object detection", viewer="opencv")

    # Press 'q' to quit the video processing
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release everything
stream.release()
out.release()
cv2.destroyAllWindows()